In [10]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder

## Ejercicio 1: Filtrado Colaborativo Basado en Ítems

Implementa un sistema de recomendación basado en ítems utilizando la matriz de calificaciones proporcionada en el ejemplo práctico 1. Recomendale al Usuario 'B' dos productos que aún no ha calificado.

In [2]:
ratings = pd.DataFrame({
    'Item1': {'A': 5, 'B': 3, 'C': 4, 'D': 4},
    'Item2': {'A': 3, 'B': 1, 'C': 3, 'D': 3},
    'Item3': {'A': 4, 'B': 3, 'C': 3, 'D': 5},
    'Item4': {'A': 3, 'B': np.nan, 'C': 5, 'D': 4},
    'Item5': {'A': 1, 'B': 5, 'C': 4, 'D': 2},
    'Item6': {'A': 4, 'B': np.nan, 'C': 5, 'D': 3}
})

print("Matriz de calificaciones original:")
print(ratings)

Matriz de calificaciones original:
   Item1  Item2  Item3  Item4  Item5  Item6
A      5      3      4    3.0      1    4.0
B      3      1      3    NaN      5    NaN
C      4      3      3    5.0      4    5.0
D      4      3      5    4.0      2    3.0


In [3]:
# Transponer la matriz para tener ítems como filas
item_ratings = ratings.T

# Calcular similitud entre ítems usando similitud del coseno
item_similarity = pd.DataFrame(
    cosine_similarity(item_ratings.fillna(0)),
    index=item_ratings.index,
    columns=item_ratings.index
)

print("\nMatriz de similitud entre ítems:")
print(item_similarity)


Matriz de similitud entre ítems:
          Item1     Item2     Item3     Item4     Item5     Item6
Item1  1.000000  0.977008  0.977534  0.887796  0.798549  0.905204
Item2  0.977008  1.000000  0.959532  0.962140  0.724462  0.962140
Item3  0.977534  0.959532  1.000000  0.865340  0.787008  0.846929
Item4  0.887796  0.962140  0.865340  1.000000  0.646395  0.980000
Item5  0.798549  0.724462  0.787008  0.646395  1.000000  0.625543
Item6  0.905204  0.962140  0.846929  0.980000  0.625543  1.000000


In [4]:
# Identificar ítems que el Usuario B no ha calificado
unrated_items = ratings.columns[ratings.loc['B'].isna()].tolist()
print(f"\nÍtems no calificados por el Usuario B: {unrated_items}")

# Calcular predicciones para los ítems no calificados
predictions = {}


Ítems no calificados por el Usuario B: ['Item4', 'Item6']


In [5]:
for item in unrated_items:
    # Obtener ítems que B ha calificado
    rated_items = ratings.columns[~ratings.loc['B'].isna()].tolist()
    
    # Calcular predicción ponderada
    numerator = 0
    denominator = 0
    
    for rated_item in rated_items:
        # Obtener calificación del usuario para el ítem calificado
        rating = ratings.loc['B', rated_item]
        # Obtener similitud entre el ítem no calificado y el ítem calificado
        similarity = item_similarity.loc[item, rated_item]
        
        numerator += similarity * rating
        denominator += abs(similarity)
    
    # Calcular predicción
    if denominator > 0:
        predictions[item] = numerator / denominator
    else:
        predictions[item] = 0

print("\nPredicciones para ítems no calificados:")
for item, prediction in predictions.items():
    print(f"{item}: {prediction:.2f}")


Predicciones para ítems no calificados:
Item4: 2.81
Item6: 2.80


In [6]:
recommendations = sorted(predictions.items(), key=lambda x: x[1], reverse=True)[:2]
print("\nRecomendaciones para el Usuario B:")
for item, score in recommendations:
    print(f"{item} (puntuación predicha: {score:.2f})")


Recomendaciones para el Usuario B:
Item4 (puntuación predicha: 2.81)
Item6 (puntuación predicha: 2.80)


## Ejercicio 2: Evaluación de Métricas
Dada la siguiente matriz de calificaciones reales y predichas para un conjunto de usuarios y productos, calcula el RMSE y el MAE.

|Usuario|	Producto1 (Real)|	Producto1 (Pred)|	Producto2 (Real)|	Producto2 (Pred)|
|---|---|---|---|---|
|A|	5|	4.5|	3|	3.5|
|B|	2|	2.0|	4|	3.8|
|C|	0|	1.2|	5|	4.9|
|D|	3|	3.5|	0|	0.2|

In [ ]:
data = {
    'Usuario': ['A', 'B', 'C', 'D'],
    'Producto1_Real': [5, 2, 0, 3],
    'Producto1_Pred': [4.5, 2.0, 1.2, 3.5],
    'Producto2_Real': [3, 4, 5, 0],
    'Producto2_Pred': [3.5, 3.8, 4.9, 0.2]
}

df = pd.DataFrame(data)
print("Datos originales:")
print(df)

# Extraer valores reales y predichos
y_true = np.array(df[['Producto1_Real', 'Producto2_Real']]).flatten()
y_pred = np.array(df[['Producto1_Pred', 'Producto2_Pred']]).flatten()

# Calcular RMSE
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
# Calcular MAE
mae = mean_absolute_error(y_true, y_pred)

print("\nValores reales:", y_true)
print("Valores predichos:", y_pred)
print("\nRMSE (Root Mean Square Error):", round(rmse, 4))
print("MAE (Mean Absolute Error):", round(mae, 4))

Datos originales:
  Usuario  Producto1_Real  Producto1_Pred  Producto2_Real  Producto2_Pred
0       A               5             4.5               3             3.5
1       B               2             2.0               4             3.8
2       C               0             1.2               5             4.9
3       D               3             3.5               0             0.2

Valores reales: [5 3 2 4 0 5 3 0]
Valores predichos: [4.5 3.5 2.  3.8 1.2 4.9 3.5 0.2]

RMSE (Root Mean Square Error): 0.5339
MAE (Mean Absolute Error): 0.4


In [9]:
print("\nCálculo detallado:")
errors = y_true - y_pred
squared_errors = errors**2
absolute_errors = np.abs(errors)

print("Errores:", errors)
print("Errores cuadrados:", squared_errors)
print("Errores absolutos:", absolute_errors)

print(f"\nRMSE = sqrt(mean(squared_errors)) = sqrt({np.mean(squared_errors):.4f}) = {rmse:.4f}")
print(f"MAE = mean(absolute_errors) = {np.mean(absolute_errors):.4f}")


Cálculo detallado:
Errores: [ 0.5 -0.5  0.   0.2 -1.2  0.1 -0.5 -0.2]
Errores cuadrados: [0.25 0.25 0.   0.04 1.44 0.01 0.25 0.04]
Errores absolutos: [0.5 0.5 0.  0.2 1.2 0.1 0.5 0.2]

RMSE = sqrt(mean(squared_errors)) = sqrt(0.2850) = 0.5339
MAE = mean(absolute_errors) = 0.4000


### Ejercicio 3: Filtrado Basado en Contenido Avanzado
Ampliando el ejemplo práctico 2, incluye una nueva columna que clasifique los productos en categorías (por ejemplo, 'Electrónica', 'Cocina', 'Literatura', 'Moda', 'Juguetes'). Utiliza esta información para mejorar el sistema de recomendación basado en contenido, de manera que solo recomiende productos de la misma categoría que el producto objetivo.

In [11]:
data = {
    'ProductID': ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8'],
    'Precio': [150, 300, 80, 450, 200, 90, 120, 350],
    'Tamaño': [2, 4, 1, 5, 3, 1, 2, 4],
    'Potencia': [3, 5, 2, 5, 4, 2, 3, 5],
    'Categoría': ['Electrónica', 'Electrónica', 'Cocina', 'Electrónica', 
                  'Cocina', 'Literatura', 'Juguetes', 'Moda']
}

df = pd.DataFrame(data)
print("Dataset original de productos:")
print(df)

Dataset original de productos:
  ProductID  Precio  Tamaño  Potencia    Categoría
0        P1     150       2         3  Electrónica
1        P2     300       4         5  Electrónica
2        P3      80       1         2       Cocina
3        P4     450       5         5  Electrónica
4        P5     200       3         4       Cocina
5        P6      90       1         2   Literatura
6        P7     120       2         3     Juguetes
7        P8     350       4         5         Moda


In [13]:
numeric_features = ['Precio', 'Tamaño', 'Potencia']
categorical_features = ['Categoría']

# Normalizar características numéricas
df_numeric = df[numeric_features].copy()
for feature in numeric_features:
    df_numeric[feature] = (df_numeric[feature] - df_numeric[feature].min()) / (df_numeric[feature].max() - df_numeric[feature].min())

# Codificar características categóricas usando one-hot encoding
encoder = OneHotEncoder(sparse_output=False)
encoded_categories = encoder.fit_transform(df[categorical_features])
category_names = [f"Cat_{cat}" for cat in encoder.categories_[0]]
df_categorical = pd.DataFrame(encoded_categories, columns=category_names, index=df.index)

# Combinar características numéricas y categóricas
df_features = pd.concat([df_numeric, df_categorical], axis=1)
print("\nCaracterísticas normalizadas y codificadas:")
print(df_features)


Características normalizadas y codificadas:
     Precio  Tamaño  Potencia  Cat_Cocina  Cat_Electrónica  Cat_Juguetes  \
0  0.189189    0.25  0.333333         0.0              1.0           0.0   
1  0.594595    0.75  1.000000         0.0              1.0           0.0   
2  0.000000    0.00  0.000000         1.0              0.0           0.0   
3  1.000000    1.00  1.000000         0.0              1.0           0.0   
4  0.324324    0.50  0.666667         1.0              0.0           0.0   
5  0.027027    0.00  0.000000         0.0              0.0           0.0   
6  0.108108    0.25  0.333333         0.0              0.0           1.0   
7  0.729730    0.75  1.000000         0.0              0.0           0.0   

   Cat_Literatura  Cat_Moda  
0             0.0       0.0  
1             0.0       0.0  
2             0.0       0.0  
3             0.0       0.0  
4             0.0       0.0  
5             1.0       0.0  
6             0.0       0.0  
7             0.0       1.0  


In [14]:
# Calcular matriz de similitud del coseno
similarity_matrix = pd.DataFrame(
    cosine_similarity(df_features),
    index=df['ProductID'],
    columns=df['ProductID']
)
print("\nMatriz de similitud entre productos:")
print(similarity_matrix)


Matriz de similitud entre productos:
ProductID        P1        P2        P3        P4        P5        P6  \
ProductID                                                               
P1         1.000000  0.869741  0.000000  0.805891  0.276950  0.004648   
P2         0.869741  1.000000  0.000000  0.979302  0.538896  0.009407   
P3         0.000000  0.000000  1.000000  0.000000  0.745432  0.000000   
P4         0.805891  0.979302  0.000000  1.000000  0.555717  0.013509   
P5         0.276950  0.538896  0.745432  0.555717  1.000000  0.006532   
P6         0.004648  0.009407  0.000000  0.013509  0.006532  1.000000   
P7         0.162086  0.314724  0.000000  0.317550  0.261747  0.002683   
P8         0.340562  0.664536  0.000000  0.704764  0.541655  0.011207   

ProductID        P7        P8  
ProductID                      
P1         0.162086  0.340562  
P2         0.314724  0.664536  
P3         0.000000  0.000000  
P4         0.317550  0.704764  
P5         0.261747  0.541655  
P6     

In [15]:
def recommend_same_category(product_id, similarity_matrix, df, top_n=3):
    # Obtener la categoría del producto objetivo
    target_category = df.loc[df['ProductID'] == product_id, 'Categoría'].values[0]
    
    # Filtrar productos de la misma categoría (excluyendo el producto objetivo)
    same_category_products = df[df['Categoría'] == target_category]
    same_category_products = same_category_products[same_category_products['ProductID'] != product_id]
    
    if same_category_products.empty:
        return "No hay otros productos en la misma categoría."
    
    # Obtener similitudes para productos de la misma categoría
    similarities = similarity_matrix.loc[product_id, same_category_products['ProductID']]
    
    # Ordenar por similitud y obtener los top_n
    recommendations = similarities.sort_values(ascending=False).head(top_n)
    
    return recommendations

In [16]:
# Probar el sistema de recomendación para diferentes productos
test_products = ['P1', 'P3', 'P6']

In [17]:
print("\nRecomendaciones basadas en contenido (misma categoría):")
for product in test_products:
    category = df.loc[df['ProductID'] == product, 'Categoría'].values[0]
    print(f"\nPara {product} ({category}):")

    recommendations = recommend_same_category(product, similarity_matrix, df)

    if isinstance(recommendations, str):
        print(recommendations)
    else:
        for prod, sim in recommendations.items():
            print(f"  - {prod} ({df.loc[df['ProductID'] == prod, 'Categoría'].values[0]}): {sim:.4f}")


Recomendaciones basadas en contenido (misma categoría):

Para P1 (Electrónica):
  - P2 (Electrónica): 0.8697
  - P4 (Electrónica): 0.8059

Para P3 (Cocina):
  - P5 (Cocina): 0.7454

Para P6 (Literatura):
No hay otros productos en la misma categoría.
